In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [5]:
# Generar datos para el módulo de llevadas
def generate_carry_data():
    x_data = []
    y_data = []
    for a in range(10):  # Primer número
        for b in range(10):  # Segundo número
            x_data.append([a, b])  # Entrada
            y_data.append(1 if (a + b) >= 10 else 0)
    return np.array(x_data), np.array(y_data)

# Generar datos
x, y = generate_carry_data()

# Convertir etiquetas a one-hot (aunque solo haya 2 clases, 0 y 1)
y_one_hot = to_categorical(y, num_classes=2)

# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
train_size = int(len(x) * 0.8)  # 80% de datos para entrenamiento
x_train, y_train = x[:train_size], y_one_hot[:train_size]
x_val, y_val = x[train_size:], y_one_hot[train_size:]  # 20% para validación

# Asegurar formas consistentes para las entradas
x_train = np.expand_dims(x_train, axis=1)  # Agrega una dimensión para 'timesteps'
x_val = np.expand_dims(x_val, axis=1)      # Haz lo mismo con los datos de validación

# Construir el modelo
model = Sequential([
    LSTM(16, input_shape=(1, 2), return_sequences=True),
    LSTM(32, return_sequences=False),
    Dense(2, activation='softmax')  # Salida para 2 clases: 0 (sin llevada) o 1 (con llevada)
])

# Callback personalizado para detener cuando todas las combinaciones sean correctas
class StopWhenPerfectCallback(EarlyStopping):
    def __init__(self, val_data, **kwargs):
        super().__init__(**kwargs)
        self.val_data = val_data  # Pasamos explícitamente los datos de validación
    
    def on_epoch_end(self, epoch, logs=None):
        # Evaluar el rendimiento sobre todo el conjunto de validación
        val_predictions = self.model.predict(self.val_data[0])
        val_pred_labels = np.argmax(val_predictions, axis=1)
        val_true_labels = np.argmax(self.val_data[1], axis=1)
        
        # Verificar si todas las combinaciones son correctas
        correct_predictions = np.sum(val_pred_labels == val_true_labels)
        total_predictions = len(val_true_labels)
        
        print(f'Evaluación de validación: {correct_predictions}/{total_predictions} correctas')

        # Si todas las combinaciones son correctas, detener el entrenamiento
        if correct_predictions == total_predictions:
            print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
            self.model.stop_training = True

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear una instancia del callback con los datos de validación
stop_callback = StopWhenPerfectCallback(val_data=(x_val, y_val), patience=500)

# Entrenar el modelo con el callback personalizado
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    batch_size=1,
    callbacks=[stop_callback]
)

# Evaluación final
loss, accuracy = model.evaluate(x_val, y_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Predicciones
predictions = model.predict(x_val)
predicted_numbers = np.argmax(predictions, axis=1)
real_numbers = np.argmax(y_val, axis=1)  # Etiquetas reales

print("Predicciones (probabilidades):", predictions[:5])
print("Predicciones (números):", predicted_numbers[:5])
print("Etiquetas reales:", real_numbers[:5])

# Mostrar las predicciones de validación y sus etiquetas
print("\nEvaluación de validación final:")
correct_predictions = np.sum(predicted_numbers == real_numbers)
total_predictions = len(real_numbers)
print(f'Predicciones correctas: {correct_predictions}/{total_predictions}')

# Guardar el modelo
model.save('carry_module.keras')

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/stepep - accuracy: 0.6821 - loss: 0.65
Evaluación de validación: 13/20 correctas
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6838 - loss: 0.6528 - val_accuracy: 0.6500 - val_loss: 0.7306
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/steptep - accuracy: 0.9263 - loss: 0.356
Evaluación de validación: 15/20 correctas
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9239 - loss: 0.3583 - val_accuracy: 0.7500 - val_loss: 0.6873
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/stepp - accuracy: 0.9375 - loss: 0.1326
Evaluación de validación: 17/20 correctas
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9116 - loss: 0.1904 - val_accuracy: 0.8500 - val_loss: 0.3773
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/steptep - accuracy: 0.9550 - loss: 0.201
Evaluación de validación: 17/20 correctas
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9550 - loss: 0.2018 - val_accuracy: 0.8500 - val_loss: 0.3978
Epoch 5/500
1/1 ━━━━━━━━━━━━━━